In [593]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [595]:
df = pd.read_csv('client_old_new_df.csv')

In [541]:
df["client_id"].nunique()

70609

In [497]:
#df.head(50)

Completion Rate:
The proportion of users who reach the final ‘confirm’ step.

Time Spent on Each Step:
The average duration users spend on each step.

Error Rates:
If there’s a step where users go back to a previous step, it may indicate confusion or an error. Moving from a later step to an earlier one should be considered an error.

<h1>Proportions of ALL Clients and Process Step</h1>

In [533]:
# Create crosstab between 'client_id' and 'process_step'
crosstab_process = pd.crosstab(df['client_id'], df['process_step'])

# Calculate the total number of unique users
total_users_all = len(crosstab_process)

# Define the process steps
process_steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Initialize a dictionary to store the completion rates for each step
completion_rates = {}

# Iterate through each process step to calculate the completion rate
for step in process_steps:
    # Calculate the number of unique users who reached each step
    completed_users = (crosstab_process[step] > 0).sum()  # Count unique users who reached the step
    completion_rate = completed_users / total_users_all if total_users_all > 0 else 0  # Prevent division by zero
    completion_rates[step] = completion_rate * 100  # Store the completion rate as a percentage

# Print the completion rates for all steps
for step, rate in completion_rates.items():
    print(f"Completion Rate for '{step}': {rate:.2f}%")

# Calculate the overall completion rate for the 'confirm' step
overall_completion_rate = completion_rates['confirm']
print(f"Overall Completion Rate for 'confirm': {overall_completion_rate:.2f}%")


Completion Rate for 'start': 99.16%
Completion Rate for 'step_1': 87.96%
Completion Rate for 'step_2': 81.01%
Completion Rate for 'step_3': 75.85%
Completion Rate for 'confirm': 67.57%
Overall Completion Rate for 'confirm': 67.57%


In [535]:
crosstab_process['confirm'].shape

(50500,)

<h1>Proportions of TEST Clients and Process Step</h1>

In [284]:
# Filter the DataFrame to include only rows where 'Variation' == 'Test'
df_filtered_yes = df[df['Variation'] == 'Test']

# Create a crosstab between 'client_id' and 'process_step' for the filtered users
crosstab_yes = pd.crosstab(df_filtered_yes['client_id'], df_filtered_yes['process_step'])

# Calculate the total number of unique users in the filtered data
total_users = len(crosstab_yes)

# Define the process steps
process_steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Initialize a dictionary to store the completion rates
completion_rates = {}

# Iterate through each process step to calculate the completion rate
for step in process_steps:
    # Calculate the number of unique users who reached each step
    completed_users = (crosstab_yes[step] > 0).sum()  # Count unique users who reached the step
    completion_rate = completed_users / total_users if total_users > 0 else 0  # Prevent division by zero
    completion_rates[step] = completion_rate * 100  # Store the completion rate as a percentage

# Print the completion rates for all steps
for step, rate in completion_rates.items():
    print(f"Completion Rate for yes '{step}': {rate:.2f}%")


Completion Rate for yes 'start': 98.93%
Completion Rate for yes 'step_1': 89.98%
Completion Rate for yes 'step_2': 82.53%
Completion Rate for yes 'step_3': 77.43%
Completion Rate for yes 'confirm': 69.29%


<h1>Proportions of NON TEST Clients and Process Step</h1>

In [286]:
# Filter the DataFrame to include only rows where 'Variation' == 'Control
df_filtered_yes = df[df['Variation'] == 'Control']

# Create a crosstab between 'client_id' and 'process_step' for the filtered users
crosstab_yes = pd.crosstab(df_filtered_yes['client_id'], df_filtered_yes['process_step'])

# Calculate the total number of unique users in the filtered data
total_users = len(crosstab_yes)

# Define the process steps
process_steps = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Initialize a dictionary to store the completion rates
completion_rates = {}

# Iterate through each process step to calculate the completion rate
for step in process_steps:
    # Calculate the number of unique users who reached each step
    completed_users = (crosstab_yes[step] > 0).sum()  # Count unique users who reached the step
    completion_rate = completed_users / total_users if total_users > 0 else 0  # Prevent division by zero
    completion_rates[step] = completion_rate * 100  # Store the completion rate as a percentage

# Print the completion rates for all steps
for step, rate in completion_rates.items():
    print(f"Completion Rate for no '{step}': {rate:.2f}%")


Completion Rate for no 'start': 99.43%
Completion Rate for no 'step_1': 85.64%
Completion Rate for no 'step_2': 79.25%
Completion Rate for no 'step_3': 74.04%
Completion Rate for no 'confirm': 65.59%


<h1>Error Rate All User</h1>

#Error Rates: If there’s a step where users go back to a previous step, it may indicate confusion or an error. 
#Moving from a later step to an earlier one should be considered an error.

In [597]:
# Convert date_time column to datetime format
df['date_time'] = pd.to_datetime(df['date_time'])

#df = df.dropna()
# Manually define the mapping dictionary for 'process_step'
# Assigning specific numeric values to each step
mapping = {
    'start': 0,  
    'step_1': 1,  
    'step_2': 2,
    'step_3' : 3,
    'confirm' : 4
}

# Map the 'process_step' column using the defined mapping
df['process_step_numeric'] = df['process_step'].map(mapping)

# Sort the dataframe by client_id and date_time
df = df.sort_values(by=['client_id', 'date_time'])

# Create a new column to track if the user moved to a previous step
df['error'] = df.groupby('client_id')['process_step_numeric'].diff().apply(lambda x: x < 0)

# Filter rows where an error occurred
error_rows = df[df['error'] == True]

# Calculate the total error rate (number of errors / total steps)
total_errors = error_rows.shape[0]
total_steps = df.shape[0]
error_rate = total_errors / total_steps

print(f"Total Errors: {total_errors}")
print(f"Error Rate: {error_rate:.2%}")

## Total Errors are more than test + control errors because of NaN values, when dropping NaN values they add up to the same

Total Errors: 45351
Error Rate: 10.08%


In [546]:
df['client_id'].nunique()

70609

<h1>Average Duration by User for Steps</h1>

In [612]:
# Time Spent on Each Step: The average duration users spend on each step.

df['date_time'] = pd.to_datetime(df['date_time'])

# Create separate columns for date and time
#df['date'] = df['date_time'].dt.date
#df['time'] = df['date_time'].dt.time
#df = df.drop('date_time', axis=1)
#df = df[["client_id","visitor_id","visit_id","Variation","test","date","time"]]

df = df.sort_values(by=['client_id', 'date_time'])

df = df.sort_values(by=['client_id', 'visitor_id', 'visit_id', 'date_time'])

# Calculate the time difference between each step within each visit
df['duration'] = df.groupby(['client_id', 'visitor_id', 'visit_id'])['date_time'].diff().dt.total_seconds().abs()

avg_step_duration_clients = df.groupby('client_id')['duration'].mean().reset_index()
avg_step_duration_clients


,client_id,duration
0,169,53.250000
1,555,39.500000
2,647,94.250000
3,722,74.875000
4,934,47.333333
...,...,...
70604,9999400,29.750000
70605,9999626,8.000000
70606,9999729,67.500000
70607,9999832,8.000000


In [614]:
avg_step_duration_total = df.groupby('process_step')['duration'].mean().reset_index()
avg_step_duration_total

,process_step,duration
0,confirm,128.396520
1,start,149.762496
2,step_1,40.272115
3,step_2,43.590126
4,step_3,95.397470


<h1>Average Duration by Control user for Steps</h1>

In [618]:
#df["test"].unique()
df_test = df[df['Variation'] != 'Control']
avg_step_duration_mean_test = df_test.groupby('process_step')['duration'].mean().reset_index()
print(avg_step_duration_mean_test)
avg_step_duration_median_test = df_test.groupby('process_step')['duration'].median().reset_index()
print(avg_step_duration_median_test)
avg_step_duration_max_test = df_test.groupby('process_step')['duration'].max().reset_index()
print(avg_step_duration_max_test)
avg_step_duration_min_test = df_test.groupby('process_step')['duration'].min().reset_index()
print(avg_step_duration_min_test)
df_test.shape

  process_step    duration
0      confirm  128.278288
1        start  149.082406
2       step_1   39.051832
3       step_2   45.965453
4       step_3   96.631742
  process_step  duration
0      confirm      59.0
1        start      52.0
2       step_1      12.0
3       step_2      24.0
4       step_3      66.0
  process_step  duration
0      confirm   14581.0
1        start   24819.0
2       step_1    3636.0
3       step_2    1858.0
4       step_3    2668.0
  process_step  duration
0      confirm       0.0
1        start       0.0
2       step_1       0.0
3       step_2       0.0
4       step_3       0.0


,client_id,visitor_id,visit_id,process_step,date_time,Variation,process_step_numeric,error,duration
280826,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36,NaN,0,False,NaN
280825,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45,NaN,1,False,9.0
280824,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31,NaN,2,False,46.0
280823,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05,NaN,3,False,94.0
280822,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09,NaN,4,False,64.0
...,...,...,...,...,...,...,...,...,...
269918,9999839,948322592_28990736246,715530422_68620416793_515645,start,2017-03-29 12:11:32,NaN,0,False,157.0
269917,9999839,948322592_28990736246,715530422_68620416793_515645,step_1,2017-03-29 12:11:35,NaN,1,False,3.0
269916,9999839,948322592_28990736246,715530422_68620416793_515645,step_2,2017-03-29 12:11:50,NaN,2,False,15.0
269915,9999839,948322592_28990736246,715530422_68620416793_515645,step_3,2017-03-29 12:12:28,NaN,3,False,38.0


<h1>Average Duration by Test user for Steps</h1>

In [620]:
df_no_test = df[df['Variation'] != 'Test']
avg_step_duration_mean_no_test = df_no_test.groupby('process_step')['duration'].mean().reset_index()
print(avg_step_duration_mean_no_test)
avg_step_duration_median_no_test = df_no_test.groupby('process_step')['duration'].median().reset_index()
print(avg_step_duration_median_no_test)
avg_step_duration_max_no_test = df_no_test.groupby('process_step')['duration'].max().reset_index()
print(avg_step_duration_max_no_test)
avg_step_duration_min_no_test = df_no_test.groupby('process_step')['duration'].min().reset_index()
print(avg_step_duration_min_no_test)
df_no_test.shape

  process_step    duration
0      confirm  128.139205
1        start  151.642459
2       step_1   42.016859
3       step_2   40.696009
4       step_3   94.509760
  process_step  duration
0      confirm      68.0
1        start      43.0
2       step_1      15.0
3       step_2      21.0
4       step_3      66.0
  process_step  duration
0      confirm    9396.0
1        start   40235.0
2       step_1    4150.0
3       step_2    1724.0
4       step_3    3635.0
  process_step  duration
0      confirm       0.0
1        start       0.0
2       step_1       0.0
3       step_2       0.0
4       step_3       0.0


(271984, 9)

<h1>Error Rate TEST User</h1>

In [573]:
# Convert date_time column to datetime format
df['date_time'] = pd.to_datetime(df['date_time'])

# Filter the dataframe for users with 'Test' in the 'Variation' column
filtered_test = df[df['Variation'] == 'Test'].copy()

# Manually define the mapping dictionary for 'process_step'
mapping = {
    'start': 0,  
    'step_1': 1,  
    'step_2': 2,
    'step_3': 3,
    'confirm': 4
}

# Map the 'process_step' column using the defined mapping
filtered_test['process_step_numeric'] = filtered_test['process_step'].map(mapping)

# Sort the filtered dataframe by client_id and date_time
filtered_test = filtered_test.sort_values(by=['client_id', 'date_time'])

# Create a new column to track if the user moved to a previous step
filtered_test['error'] = filtered_test.groupby('client_id')['process_step_numeric'].diff().apply(lambda x: x < 0)

# Filter rows where an error occurred
error_rows = filtered_test[filtered_test['error'] == True]

# Calculate the total error rate (number of errors / total steps)
test_errors = error_rows.shape[0]
test_steps = filtered_test.shape[0]  # Use filtered dataframe for total steps
test_error_rate = test_errors / test_steps if test_steps > 0 else 0  # Avoid division by zero

print(f"Total Test Errors: {test_errors}")
print(f"Test Error Rate: {test_error_rate:.2%}")


Total Test Errors: 19088
Test Error Rate: 10.73%


<h1>Error Rate CONTROL User</h1>

In [588]:
# Convert date_time column to datetime format
df['date_time'] = pd.to_datetime(df['date_time'])

# Filter the dataframe for users with 'Test' in the 'Variation' column
filtered_control = df[df['Variation'] == 'Control'].copy()

# Manually define the mapping dictionary for 'process_step'
mapping = {
    'start': 0,  
    'step_1': 1,  
    'step_2': 2,
    'step_3': 3,
    'confirm': 4
}

# Map the 'process_step' column using the defined mapping
filtered_control['process_step_numeric'] = filtered_control['process_step'].map(mapping)

# Sort the filtered dataframe by client_id and date_time
filtered_control = filtered_control.sort_values(by=['client_id', 'date_time'])

# Create a new column to track if the user moved to a previous step
filtered_control['error'] = filtered_control.groupby('client_id')['process_step_numeric'].diff().apply(lambda x: x < 0)

# Filter rows where an error occurred
error_rows = filtered_control[filtered_control['error'] == True]

# Calculate the total error rate (number of errors / total steps)
control_errors = error_rows.shape[0]
control_steps = filtered_control.shape[0]  # Use filtered dataframe for total steps
control_error_rate = control_errors / control_steps if test_steps > 0 else 0  # Avoid division by zero

print(f"Total Control Errors: {control_errors}")
print(f"Control Error Rate: {control_error_rate:.2%}")


Total Control Errors: 13335
Control Error Rate: 9.30%
